In [18]:
import preprocessing as prep
import numpy as np
import pandas as pd
import nltk
import string
import pickle
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(42)

import keras
import pickle
import sklearn
import analysis_util as util
from sklearn.metrics import roc_curve, auc, plot_roc_curve
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from keras import metrics, models, layers, optimizers

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/raw.csv', encoding='latin1')

In [3]:
df.columns = ['tweet', 'product', 'emotion']

In [4]:
df = df[df['emotion']!="I can't tell"]
df.dropna(axis=0, subset=['tweet'], inplace=True)

In [5]:
X = df['tweet']
y = df['emotion']

In [6]:
df = pd.concat([X,y],axis=1)

In [7]:
df

,tweet,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion
...,...,...
9088,Ipad everywhere. #SXSW {link},Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,No emotion toward brand or product


In [8]:
X_train, X_test, y_train, y_test = prep.split(df, int(len(df)*.15))
X_train, X_train_val, y_train, y_train_val = prep.split(pd.concat([X_train, y_train], axis=1), int(len(df)*.15))

In [9]:
X_train = X_train.reset_index()['tweet']
X_train_val = X_train_val.reset_index()['tweet']
X_test = X_test.reset_index()['tweet']

In [10]:
y_train = y_train.reset_index()['emotion']
y_train_val = y_train_val.reset_index()['emotion']
y_test = y_test.reset_index()['emotion']

In [11]:
X_train = list(map(prep.preprocess, list(X_train)))
X_train_val = list(map(prep.preprocess, list(X_train_val)))
X_test = list(map(prep.preprocess, list(X_test)))

In [12]:
X_train_tokenized = list(map(word_tokenize, X_train))
X_train_val_tokenized = list(map(word_tokenize, X_train_val))
X_test_tokenized = list(map(word_tokenize, X_test))

In [13]:
w2v_model = Word2Vec(X_train_tokenized, size=100, window=5, min_count=1, workers=4)

In [14]:
w2v_model.wv.most_similar(positive='apple')

[('shop', 0.9994764924049377),
 ('pop', 0.9984753131866455),
 ('open', 0.9978106617927551),
 ('temp', 0.9976313710212708),
 ('core', 0.9967163801193237),
 ('dt', 0.995648205280304),
 ('set', 0.995041012763977),
 ('congress', 0.9946262240409851),
 ('austin', 0.9938600063323975),
 ('sixth', 0.992957353591919)]

In [15]:
X_train_w2v = prep.w2v_vectorize(w2v_model.wv, X_train_tokenized)
X_train_val_w2v = prep.w2v_vectorize(w2v_model.wv, X_train_val_tokenized)
X_test_w2v = prep.w2v_vectorize(w2v_model.wv, X_test_tokenized)

In [19]:
clfs = [LogisticRegressionCV(cv=5), 
        RandomForestClassifier(n_estimators=100), 
        SVC()]

In [21]:
for clf in clfs:
    clf.fit(X_train_w2v, y_train)
    print(f'{clf}')
    print('--Train--')
    util.report(y_train, clf.predict(X_train_w2v))
    print('--Validation--')
    util.report(y_train_val, clf.predict(X_train_val_w2v))

/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

LogisticRegressionCV(cv=5)
--Train--
[[ 0.  86.1 13.9]
 [ 0.  91.7  8.3]
 [ 0.  75.8 24.2]]

                                    precision    recall  f1-score   support

                  Negative emotion       0.00      0.00      0.00       389
No emotion toward brand or product       0.64      0.92      0.76      3767
                  Positive emotion       0.58      0.24      0.34      2100

                          accuracy                           0.63      6256
                         macro avg       0.41      0.39      0.37      6256
                      weighted avg       0.58      0.63      0.57      6256

--Validation--
[[ 0.  84.7 15.3]
 [ 0.  91.1  8.9]
 [ 0.  75.1 24.9]]

                                    precision    recall  f1-score   support

                  Negative emotion       0.00      0.00      0.00        85
No emotion toward brand or product       0.64      0.91      0.75       797
                  Positive emotion       0.58      0.25      0.35       

/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

[[  0. 100.   0.]
 [  0. 100.   0.]
 [  0. 100.   0.]]

                                    precision    recall  f1-score   support

                  Negative emotion       0.00      0.00      0.00        85
No emotion toward brand or product       0.59      1.00      0.75       797
                  Positive emotion       0.00      0.00      0.00       458

                          accuracy                           0.59      1340
                         macro avg       0.20      0.33      0.25      1340
                      weighted avg       0.35      0.59      0.44      1340



/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/stlanier/opt/anaconda3/envs/flatiron/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo